# Title

## Introduction 

Stars are large spheres of hot gas that emit heat and light into space. They are composed of mostly hydrogen, with some helium and other elements. The sun is an example of a star and is the closest star to Earth (NASA).

Galaxies are clusters of planets, stars, gasses, and dust that are all held together by gravity. Galaxies are very large and emit light from the stars and other things that it contains. The Milky Way Galaxy is an example of a galaxy and is the one that Earth is a part of (NASA).

Quasars are the core of active galaxies and they are powered by supermassive black holes. They emit immense amounts of heat and light due to the friction of material being drawn in. The closest quasar to Earth is called 3C 273 and can be seen with an 8-inch telescope (Space.com).

The classification of celestial objects into sta
rs, galaxies, and quasars has been pivotal for the understanding of planet Earth's positioning within space. It has led to key insights such as the discovery that the Andromeda galaxy is separate from our own, and this classification continues to be essential for astrological research (Clarke 2019).

In this report, we will use data on celestial objects to answer the following question: "Based on its redshift and brightness in different wavelengths of light, what type of celestial object is this?" 

Our data set is from Sloan Digital Sky Survey Data Release 16. It was collected by the Sloan Digital Sky Survey Telescope; a powerful telescope aimed at measuring spectral characteristics. It contains data on light emitted from galaxies, quasars, and stars, including redshift, which reflects how quickly an object moves (SDSS Voyages, 2024b), and brightness in five wavelengths of light. Below are a list of the variables collected as well as what they represent:

* obj_ID = Object Identifier, the unique value that identifies the object in the image catalog used by the CAS
* alpha = Right Ascension angle
* delta = Declination angle
* u = Ultraviolet filter in the photometric system
* g = Green filter in the photometric system
* r = Red filter in the photometric system
* i = Near Infrared filter in the photometric system
* z = Infrared filter in the photometric system
* run_ID = Run Number used to identify the specific scan
* rereun_ID = Rerun Number to specify how the image was processed
* cam_col = Camera column to identify the scanline within the run
* field_ID = Field number to identify each field
* spec_obj_ID = Unique ID used for optical spectroscopic objects (this means that 2 different observations with the same spec_obj_ID must share the output class)
* class = object class (galaxy, star, or quasar object)
* redshift = redshift value based on the increase in wavelength
* plate = plate ID, identifies each plate in SDSS
* MJD = Modified Julian Date, used to indicate when a given piece of SDSS data was taken
* fiber_ID = fiber ID that identifies the fiber that pointed the light at the focal plane in each observation

We will focus on the u, g, r, i, z, and redshift variables to help predict the classification of the class variable.


## Import Libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import altair as alt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Methods and Results

In [2]:
# Load in the data file from the web
url="https://drive.google.com/file/d/1LM-kB1xP90O9RBY5yjRP1mET_BKOOhxC/view?usp=sharing"
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
star_data = pd.read_csv(url) #Citation: (Pandas, 2019)

star_data

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
0,1237666301628060000,47.372545,0.820621,18.69254,17.13867,16.55555,16.34662,16.17639,4849,301,5,771,8168632633242440000,STAR,0.000115,7255,56597,832
1,1237673706652430000,116.303083,42.455980,18.47633,17.30546,17.24116,17.32780,17.37114,6573,301,6,220,9333948945297330000,STAR,-0.000093,8290,57364,868
2,1237671126974140000,172.756623,-8.785698,16.47714,15.31072,15.55971,15.72207,15.82471,5973,301,1,13,3221211255238850000,STAR,0.000165,2861,54583,42
3,1237665441518260000,201.224207,28.771290,18.63561,16.88346,16.09825,15.70987,15.43491,4649,301,3,121,2254061292459420000,GALAXY,0.058155,2002,53471,35
4,1237665441522840000,212.817222,26.625225,18.88325,17.87948,17.47037,17.17441,17.05235,4649,301,3,191,2390305906828010000,GALAXY,0.072210,2123,53793,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1237667968033620000,228.674917,19.179460,19.32631,18.82748,18.65659,18.60481,18.60917,5237,301,5,134,4448615345201370000,QSO,0.438182,3951,55681,672
99996,1237664818210470000,173.757382,36.441603,18.33687,17.30365,17.16037,17.14895,17.14419,4504,301,2,111,2265404129658560000,STAR,-0.000497,2012,53493,340
99997,1237664295297290000,205.426531,38.499053,17.50690,15.63152,15.22328,15.04469,15.28668,4382,301,4,97,2257446413900210000,GALAXY,0.004587,2005,53472,62
99998,1237656537513130000,337.135144,-9.635967,19.33946,17.21436,16.29697,15.86745,15.51556,2576,301,2,105,811847537492257000,GALAXY,0.084538,721,52228,268


In [3]:
# Filter relevant columns 
star_filtered = (
    star_data.loc[:, ["u", "g", "r", "i", "z", "redshift", "class"]]
    .rename(columns={
        "u":"Ultraviolet", 
        "g":"Green", 
        "r":"Red", 
        "i":"Near Infrared", 
        "z":"Infrared",
        "redshift":"Redshift",
        "class":"Class"
    })
)
star_filtered

,Ultraviolet,Green,Red,Near Infrared,Infrared,Redshift,Class
0,18.69254,17.13867,16.55555,16.34662,16.17639,0.000115,STAR
1,18.47633,17.30546,17.24116,17.32780,17.37114,-0.000093,STAR
2,16.47714,15.31072,15.55971,15.72207,15.82471,0.000165,STAR
3,18.63561,16.88346,16.09825,15.70987,15.43491,0.058155,GALAXY
4,18.88325,17.87948,17.47037,17.17441,17.05235,0.072210,GALAXY
...,...,...,...,...,...,...,...
99995,19.32631,18.82748,18.65659,18.60481,18.60917,0.438182,QSO
99996,18.33687,17.30365,17.16037,17.14895,17.14419,-0.000497,STAR
99997,17.50690,15.63152,15.22328,15.04469,15.28668,0.004587,GALAXY
99998,19.33946,17.21436,16.29697,15.86745,15.51556,0.084538,GALAXY


## Discussion

## References

* https://www.kaggle.com/datasets/fedesoriano/stellar-classification-dataset-sdss17/data
* https://www.sdss4.org/dr17/
* https://www.sdss4.org/instruments/camera/#Filters
* https://science.nasa.gov/universe/stars/
* https://www.aanda.org/articles/aa/full_html/2020/07/aa36770-19/aa36770-19.html#R16
* https://www.space.com/17262-quasar-definition.html
* https://science.nasa.gov/universe/stars/
* https://science.nasa.gov/universe/galaxies/